# ***Τροπος Επεξηγησης Κωδικα***
**Η επεξηγηση του κωδικα εχει γινει με τετοια μορφη που να βγαζει νοημα απο αποψη εκτελεσης.** 

Δεν εχω παει απλα να πω τι κανει ενα block code, αλλα πηγαινω νοηματικα απο σημειο σε σημειο που πιστευω οτι λειτουργουν συνεργατικα. Επομενως υπαρχει περιπτωση να κανω "μεταπηδημα" στις εξηγησεις μου , αλλα πιστευω οτι αυτο ειναι το σωστο εδω λογω του τροπου εκτελεσης της python.

Θεωρω οτι θα ηταν κακο να παω να εξηγω ενα τμημα κωδικα πριν καν φανει ο τροπος εκτελεσης του αργοτερα.


# ***Φορτωση Βιβλιοθηκων***
Φορτώνουμε στο πρόγραμμα τις εξής βιβλιοθήκες
1)  requests
2)  BeautifulSoup
3)  re



### 1. Requests
**Είναι βιβλιοθήκη η οποία  χειρίζεται την επικοινωνία  του υπολογιστή  και του http server**

Συγκεκριμένα μέσω του κώδικα μας κάνουμε αιτήματα (  χρησιμοποιώντας τις συναρτήσεις αυτής της βιβλιοθήκης )  στον http server προκειμένου να συνδεθούμε σε αυτόν.

### 2. BeatifulSoup
**Eίναι η βιβλιοθήκη η οποία  έχει έτοιμες συναρτήσεις  για την επεξεργασία και περιήγηση σε html αρχαία**

Aυτή η βιβλιοθήκη βρίσκεται στο πακέτο bs4 (  από τον οποίο την κάνουμε import )

Εξου και  εντολή:  `from bs4 import beautiful soup`

### 3. re
**Είναι βιβλιοθήκη η οποία μας επιτρέπει να εισάγουμε κανονικές εκφράσεις στο πρόγραμμα μας,  και να κάνουμε διάφορες λειτουργίες χρησιμοποιώντας κανονικές εκφράσεις**

In [10]:
import requests
from bs4 import BeautifulSoup
import re

# ***Δηλωση Συναρτησεων***
Ενας python κωδικας εκτελείται από πάνω προς τα κάτω, αρχιζοντας απο την αρχη του αρχειου.
Στην δικια μας περιπτωση αυτο σημαινει οτι αφού κάνει import τις προηγούμενες βιβλιοθήκες, ο κώδικας θα συναντήσει τις τρεις παρακάτω συναρτήσεις: 

1) fetch_wikipedia_page
2) extract_links
3) crawl_wikipedia

Αυτές οι συναρτήσεις έχουν σώμα,  αλλά δεν μπορούν να εκτελεστούνε( απευθείας εκεινη την ωρα )  χωρίς να καλεστούνε πρωτα.
Επομένως ο compiler με το που φτάσει σε αυτές, τις αποθηκεύει στην μνήμη έτσι ώστε να μπορούν να κληθούν αργότερα στο προγραμμα.

Αυτό είναι απαραίτητο να γίνει διότι αυτές συναρτήσεις  θα καλεστούνε  πολύ σύντομα  και ο compiler θα πρέπει να "γνωρίζει" την ύπαρξή τους, πριν εκτελεστουν.

In [ ]:
def fetch_wikipedia_page(url):
    """
    Fetches and parses a Wikipedia page given its URL.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad requests (4xx and 5xx)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None

def extract_links(soup):
    """
    Extracts and returns a list of valid Wikipedia links from a page.
    """
    base_url = "https://en.wikipedia.org"
    links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        # Filter for Wikipedia article links
        if href.startswith('/wiki/') and not re.search(r':', href):
            full_url = base_url + href
            links.append(full_url)
    return links

def crawl_wikipedia(start_url, max_depth=2, max_pages=10):
    """
    Crawls Wikipedia starting from `start_url` up to `max_depth` levels.
    """
    parsed_paragraph = []
    visited = set()
    to_visit = [(start_url, 0)]  # (URL, depth)
    result = []

    while to_visit and len(result) < max_pages:
        current_url, depth = to_visit.pop(0)
        if depth > max_depth or current_url in visited:
            continue

        print(f"Visiting: {current_url} at depth {depth}")
        soup = fetch_wikipedia_page(current_url)
        
        if soup is None:
            continue

        paragraphs = soup.find_all('p')
        #append in one list
        text = [p.text.strip() for p in paragraphs] 
        parsed_paragraph.append(" ".join(text))

        visited.add(current_url)
        result.append(current_url)

        links = extract_links(soup)
        for link in links:
            if link not in visited:
                to_visit.append((link, depth + 1))
    
    return (result, parsed_paragraph)

# ***Βημα 1***
Μετά το importing και τις δηλώσεις,  φτάνουμε στο πρώτο σημείο του κώδικα στο οποίο αρχίζει να φαίνεται η λειτουργία του web crawler( βημα 1 )


### 1) start_url
Στην μεταβλητή start_url  εισάγουμε το URL της σελίδας που θέλουμε να αρχίσουμε την διαδικασια του web crawling και σε αυτή την περίπτωση επέλεξα να αρχίσω από μία "σχετική του θέματος" σελίδα του wikipedia.

### 2) καλεσμα της συναρτησης crawl_wikipedia
Καλούμε τη συνάρτηση "crawl_wikipedia"  η οποία δέχεται παραμέτρους: 

1.  Url της αφετηριας
2.  μεταβλητή max_depth(  η οποία  δημιουργείται εκείνη την ωρα και της εκχωρείται τιμή ταυτόχρονα μεσα στην παραμετρο της συναρητησης )
3.  μεταβλητή max_pages(  η οποία  δημιουργείται εκείνη την ωρα και της εκχωρείται τιμή ταυτόχρονα μεσα στην παραμετρο της συναρητησης )


Η συνάντηση αυτή η επιστρέφει ένα tuple τιμων(  πολλές τιμές που περιλαμβάνονται σε μία δομή ) που καθε μια αποθηκευεται στις μεταβλητές crawled_pages/ paragraphs (  οι οποίες δημιουργούνται  εκείνη την ώρα  ),  όπου ο τύπος δεδομένων τους θα είναι ίδιος με την επιστρεφόμενη τιμή σε αυτές.

Mετά το κάλεσμα της συνάρτησης η μεταβλητή crawled pages  θα έχει το πλήθος( μια λιστα ) των url που συλλέχθηκαν από την συνάρτηση,  το οποίο θα χρησιμοποιηθεί αμέσως μετά προκειμένου να περιηγηθούμε την  λίστα και να εμφανίσουμε  ποια URL συλλέχτηκαν.
Επειτα μεσω for loop θα αρχικοποιησουμε μια μεταβλητη pages ( η οποια θα παρει τις τιμες της λιστας, κανοντας την να εχει τον ιδιο τυπο δεδομενων ) και θα εμφανιζουμε το περιεχομενο της για καθε καινουργιο url της λιστας


In [ ]:
start_url = "https://en.wikipedia.org/wiki/Web_scraping"
(crawled_pages, paragraphs) = crawl_wikipedia(start_url, max_depth=2, max_pages=20)
print("\nCrawled Pages:")
for page in crawled_pages:
    print(page)

# ***Εκτελεση συναρτησης crawl_wikipedia***

### `parsed_paragraph = []`
Φτιάχνουμε μια κενή λίστα με ονομα parsed_paragraph  η οποία ***σε καθε indexing της***, θα εχει ως περιεχομενο ολες τις παραγραφους καθε
HTML αρχειου.
Δηλάδη καθε indexing θα αντιστοιχει σε ενα HTML αρχειό και το περιεχομενο θα ειναι οι παραγραφοι αυτων `<p>`

### `paragraphs = soup.find_all('p')`
Η λειτουργία της συνάρτησης είναι  να αναζητεί ολα τα tags `<p>` στο html αρχειο που βρισκομαστε εκεινη την ωρα.
Η επιστρεφόμενη τιμή είναι μία λίστα αντικειμένων,  όπου κάθε αντικείμενο ειναι μια παράγραφος.
Αυτή η εντολή επειδή βρίσκεται μέσα σε καταλληλο while loop(βλεπε παρακατω) και εκτελείται επαναληπτικά για καθε html αρχειο, το περιεχομενο της θα αλλαζει καθε φορα αναλογως του html αρχειου που βρισκομαστε.

### `text = [p.text.strip() for p in paragraphs]`

Αυτη τη εντολη θα μπορουσε να γραφτει και ως εξης:
```
text = []
for p in paragraphs
text.apppend(p.text.strip())
```
Στην αρχικη συτναξη, το κανω σε μια εντολη με την μια. 
Η ολη ιδεα ειναι οτι κανουμε εναν βροχο οπου η μεταβλητη p κανει κυκλο και αποκτα σιγα σιγα καθε τιμη(τα αντικειμενα) της λιστας **paragraphs**, αλλα ταυτοχρονα το αποτελεσμα αποθηκευεται σε μια καινουργια προσωρινη λιστα με ονομα **text**...δηλαδη αντι να δηλωσω πρωτα την λιστα και μετα να κανω την for και append σε αυτη...υπαρχει συνακτικο που με αφηνει να το κανω ολο με την μια.

Απο εκει και περα, χρησιμοποιουμε τις εξης μεθοδους
1. text: μεθοδος της beatiful soup για αφαιρεση των tags απο ενα κειμενο...δηλαδη για να παρουμε μονο το περιεχομενο των tagg
2. strip: default μεθοδος της python για αφαιρεση κενων, tabs, newlines κτλ...γενικα για μορφοποιηση ενος κειμενου

### `parsed_paragraph.append(" ".join(text))`

Εφοσον εχει δημιουργηθει η προσωρινη λιστα **text** που θα ειναι και καταλληλα μορφοποιημενη( χωρις html tags και ασχετα κενα ), την κανουμε append πλεον στην αρχικη λιστα **parsed_paragraph**  που θα κρατησει ολες τις μορφοποιημενες παραγραφους απο ολα τα html
Υπενθυμιζω οτι το περιεχομενο της  λιστας text γινεται overide σε καθε επαναληψη της while, και το περιεχομεο της μετα γινεται join( μαζι με ενα κενο ) στην αρχικη λιστα. To overide δεν μας πειραζει( ισα ισα το αντιθετο ) διοτι ο σκοπος αυτης ειναι απλα να κρατησει εκεινη την ωρα τα αντικειμενα του html αρχειου σε μορφοποιημενη μορφη.

Επομενως, κανουμε join το μοφροποιημενο αντικειμενο με ενα κενο απο πισω( ετσι ωστε οταν γινει append να ξεχωρισει απο τα προηγουμενα αντικειμενα της αρχικης λιστας) και επειτα ολο αυτο το κανουμε append.

### `visited = set()` και `visited.add(current_url)`

**Αυτές οι εντολές λειτουργούν συνεργατικά μεταξύ τους,  και ειναι σημαντικο να εξηγηθούν  σαν ζευγος(  παρόλο που δεν βρίσκονται τοσο κοντά μεταξύ τους στον κωδικα )**


Η μεθοδος set() ειναι ενα ειδικό αντικείμενο  στην python που μας επιτρέπει να αποθηκεύσουμε τιμές(σε αυτο)  τις οποίες δεν θέλουμε να επαναληφθούν στο μέλλον, διοτι το αντικειμενο set αφαιρει αυτοματα τις εκχωρησεις που υπαρχουν ηδη μεσα σε αυτο. **Επομένως και η μεταβλητή visited  θα είναι τύπου (  ειδικό αντικειμένου ) set**



Στην μεταβλητή visited (  τυπου set )  θα προσθέτουμε τα url των HTML τα οποία επισκεπτόμαστε κάθε φορά( λογω while loop),  έτσι ώστε να σιγουρευτούμε ότι σε μελλοντική επανάληψη δεν θα τα επισκεφτούμε.



Αυτός ο έλεγχος  γίνεται μέσα στη while,  χρησιμοποιώντας μια συνθήκη if που οποία ελέγχει  αν το τωρινό url (   αποθηκευμένο στη μεταβλητή current_url ) βρίσκεται μέσα σε αυτή τη μεταβλητή visted( τυπου set ). 
`if depth > max_depth or current_url in visited:`
`continue`
        
        

Αν το url  βρίσκεται όντως μέσα στη μεταβλητή visited, τοτε θα εκτελεστει μια εντολη continue αγνοωντας το συγκεκριμενο βημα επαναληψης( διοτι δεν θελουμε να ελεγξουμε κατι που εχει ελεγχτει ηδη )

### `to_visit = [(start_url, 0)]` και `current_url, depth = to_visit.pop(0)`

Αρχικα φτιάχνουμε μία λίστα με όνομα **to_visit**  στην οποία θα αποθηκεύουμε  δύο διαφορετικές τιμές για κάθε index του.

1. το url που θελουμε να επεξεργαστουμε( στην αρχη το αρχικοποιουμε με το αρχικο url )
2. το βαθος του οσο αναφορα την "σειρα" του σε σχεση με τα υπολοιπα που εχει επεξεργαστει ο crawler

Σε κάθε επανάληψη κάνουμε pop από την λίστα  το πρώτο στοιχείο της( το μηδενικο ) και αποθηκεύουμε το url στην  μεταβλητή current_url και το βαθος στην μεταβλητη depth. Aυτό μπορούμε να το κάνουμε διότι η λίστα λειτουργεί ως tuple(  αποθηκεύει 2 τιμές ),  και υπάρχουν επιτρέπει την εκχώρηση ενός tuple σε δύο διαφορετικές μεταπτές ταυτόχρονα.

### **FIFO και λογως χρησης pop(0)**
Ο λόγος που κάνουμε pop το πρώτο στοιχείο κάθε φορά( το μηδεν ) από την λίστα είναι γιατί ακολουθούμε τη λογική FIFO.

Ακολουθούμε τη λογική fifo διοτι ο crawler σαν συμπεριφορά  λειτουργεί με την έννοια του αλγοριθμου BFS (  Breadth-First Search ),  όπου  αναζητούμε τα δεδομένα με βάση το βάθος τους ( το επιπεδο βαθους...δηλαδη αναζητουμε πρωτα ολο το επιπεδο και μετα παμε σε μεγαλυτερο βαθος ).

Επειδή web crawler λειτουργεί με την ίδια λογική σαν θεωρια,  και η python  αυτόματα μας επιτρέπει να κάνουμε pop το **ΤΩΡΙΝΑ** πρωτο στοιχειο απο την λιστα γραφοντας pop(0) ακομα και αν αυτο δεν ηταν το 0 αρχικα...βγάζει απόλυτο νόημα να λειτουργήσουμε με τη λογική FIFO. 


### **Πως ενημερωνεται η μεταβλητη depth**

Η μεταβλητή depth  ενημερώνονται μέσω αυτό των 2 εντολών

1. to_visit.append((link, depth + 1))

2. current_url, depth = to_visit.pop(0)

Στην ουσία όταν φτάνουμε στην εντολή `to_visit.append((link, depth + 1))`  αποθηκεύουμε στην λίστα **to_visit**  το επόμενο link προς επεξεργασία αλλά και αυξάνουμε το βάθος της μεταβλητής depth κατα 1. Εδω να επισημανουμε οτι κατα την επισκεψη σε καποιο url, μπορει να εχει στο επιπεδο του πολλαπλα url τα οποια θα πρεπει να προσπελαστουν μεσω της while...που σημαινει αυτα θα εχουν το ιδιο επιπεδο( αρα η τιμη depth για αυτα θα παραμεινει ιδια, διοτι το αρχικο depth του γονεα δεν θα αλλαξει). 

**Θα παμε σε επομενο επιπεδο depth μονο αν καποιο απο αυτα τα "πολλαπλα" url εχουν και αυτα με την σειρα τους καποια url για επεξεργασια.**

Αυτή η επαναληπτική τη διαδικασία  κάνει το depth  να αυξάνεται ολόνα και περισσότερο σαν τιμή( οσο πιο βαθια παει ο web crawler )  με αποτέλεσμα κάποια στιγμή να φτάσει  την τιμή της μεταβλητής max_depth( οπως και αν την αρχικοποιησουμε ) 



Για παραδειγμα 

το depth στην αρχη εχει την τιμη 0

Επειτα αν παει στο επομενο url θα εχει την τιμη 1, αλλα σε αυτο το url μπορει να υπαρχουν 5+ url σε αυτο το επιπεδο τα οποια θα κανει search. 


ΑΦΟΥ ΤΕΛΕΙΩΣΕ ΜΕ ΑΥΤΑ( ΛΟΓΙΚΗ BFS ) ,τοτε θα ψαξει να δει αν στα επιμερους url που εψαξε, υπαρχουν και αλλα που πρεπει να ψαξει ( υποδικνυοντας οτι θα παει σε πιο βαθυ επιπεδο )



### `print(f"Visiting: {current_url} at depth {depth}")`

αυτή η εντολή εκτελεί επαναληπτικά(  διότι βρίσκεται μέσα σε while loop)  την τωρινή κατάσταση του crawler.

Δηλαδή το url  που επεξεργάζεται  και το βάθος του(  έχω προαναφερθεί σε αυτό λεπτομερως ακριβως απο πανω )

### `soup = fetch_wikipedia_page(current_url)`

Η εντολή αυτή εκτελεί την συνάρτηση fetch_wikipedia_page()  και παίρνει ως παράμετρο το url που επεξεργαζόμαστε  εκείνη τη στιγμή.

Κατά την εκτέλεση αυτής της συνάρτησης,  γίνεται έντονη χρήση της βιβλιοθήκης request  ως μία απόπειρα να συνδεθούμε με τον http server του συγκεκριμένου url που πήραμε ως παράμετρο.

Αυτή η απόπειρα  βρίσκεται μέσα σε **try...except**  έτσι ώστε  στην περίπτωση που αποτύχει  το request(δηλαδη να αποτυχει το try) ,  να μπουμε στο except και να επιστρέψουμε ένα ειδικο αντικειμενο **None**  πίσω στο σημείο που καλέστηκε(  διότι σε μία τέτοια περίπτωση  θα θέλαμε να κάνουμε continue  στο επόμενο βήμα της  αρχικής επανάληψης ).

# ***Φορτωση Βιβλιοθηκης pandas***

Φορτώνουμε την βιβλιοθήκη εν ονόματι **pandas**  και της αλλαζουμε το ονομα τοπικά στον κώδικα μας  ως **pd**  προκειμένου να είναι πιο  εύκολη  η πρόσβαση στις μεθόδους της.

Για να μην πολυλογώ για την θεωρητική επεξήγηση, την εχω γράψει στην αναφορά αρκετά λεπτομερώς

In [ ]:
import pandas as pd

# ***Δημιουργια DataFrame***

Εδώ δημιουργούμε ένα dataframe με ονομα **df**. Tο Dataframe είναι ένα  ειδικό αντικείμενο της βιβλιοθήκης **pandas** που  μοιάζει με ένα δύσδιάστατο array. Οι μέθοδοι της βιβλιοθήκης pandas,  μας επιτρέπουν να επεξεργαστούμε και να εμφανίσουμε το  περιεχόμενο  με διαφορετικούς τρόπους.

Στην δικιά μας περίπτωση  φτιάχνουμε δύο στήλες με ονομα:
1. URL: αποθηκευουμε ολα τα url που βρηκε η προηγουμενη συναρτηση
2. Text: Οι παραγραφοι απο καθε ενα απο αυτα τα URL

Επομενως το DataFrame μας σε καθε γραμμη του θα εχει 1 στηλη για τα URL και μια που περιεχει τις παραγραφους αυτων


In [ ]:
df = pd.DataFrame({
    "URL": crawled_pages,
    "Text": paragraphs
})

# ***Δημιουργια csv με βαση το DataFrame***

Η μέθοδος **to_csv**  δημιουργεί ενα csv αρχειο με βαση το dataframe **df**

Tο όνομα αυτού του αρχείου  το ορίζουμε ως **wikipedia_data.csv**

Επειδή το dataframe  κάνε αριθμηση στις γραμμές,  θέλουμε να βάλουμε την παράμετρο **index=False**  έτσι ώστε αυτή η αριθμηση να μην αποτελεί ξεχωριστή  στηλη στο csv(  επηρεάζοντας έτσι τη συμπεριφορά του αρχείου με το προγραμμα )

Το encoding='utf-8' διασφαλίζει ότι το αρχείο CSV αποθηκεύεται με την κωδικοποίηση UTF-8, η οποία υποστηρίζει ειδικούς χαρακτήρες




In [ ]:
df.to_csv('wikipedia_data.csv', index=False, encoding='utf-8')

In [ ]:
# Specify the path to your CSV file
csv_file_path = "wikipedia_data.csv"

# Load the CSV file into a DataFrame
data = pd.read_csv(csv_file_path, header=0)

# Display the loaded data
data.head()

# Access the column as a list
column_data = data['Text'].tolist() 

# **Importing των βιβλιοθηκων για Tokenization και επεξεργασια κειμενου**

Eχει γινει σημαντικη και λεπτομερη επεξηγηση στην αναφορα σχετικα με το τι ειναι το tokenization και πως λειτουργουν αυτες οι μεθοδοι/βιβλιοθηκες.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# ***Δηλωση της συναρτησης preprocess_text και λειτουργια της***

Αυτη ειναι η συναρτηση που θα κανει την προεπεξεργασια του κειμενου...το tokenization.
Δέχεται ως παράμετρο το text

text => item => καθε γραμμη της λιστας column_data(εκτελειται επαναλπητικα η συναρτηση για καθε τιμη της λιστας),
οπου στην ουσία είναι το item που αποκτα καθε τιμη επαναληπτικα της λιστας column_data 

1) `text = re.sub(r"[^a-zA-Z\s]", "", text)`
Χρησιμοποιεί τη μέθοδο re.sub από τη βιβλιοθήκη των Regular Expressions η οποία δέχεται ως είσοδο το text(  2η παράμετρος ) στην re.sub  και στην ουσία καθαρίζει ότι δεν είναι πεζο η κεφαλαίο λατινικό γράμμα ή κενο.

`r"[^a-zA-Z\s]"`
To ^ οταν ειναι μεσα στις αγγυλες( στην αρχη ομως ) υποδικνυει οτι αν κατι δεν αρχιζει συμφωνα με την κανονικη εκγραφση που θα ακολουθησει, τοτε θα πρεπει να αντικατασταθει ( σε αυτη την περιπτωση εχουμε επιλεξει αντικατασταση με κενο )
1. r"" Δηλωνει οτι προκειται για string
2. Οι [] ορίζουν ένα σύνολο χαρακτήρων
3. [a-z] σημαίνει οποιοδήποτε πεζό γράμμα από το 'a' έως το 'z' στην αλφαβητο
4. [Α-Ζ] Όλα τα κεφαλαία γράμματα απο 'Α' εως 'Ζ'
5. \s σημαίνει whitespace

2) `tokens = word_tokenize(text)`
Εφοσον το text καθαρισε απο την προηγουμενη εντολη, τωρα μπορουμε να παμε να χωρισουμε την προταση σε λεξεις( δηλαδη σε ξεχωριστα tokens )
Το αποτελεσμα της μεθοδου( με εισοδο την καθε καθαρισμενη πλεον παραγραφο ), αποθηκευεται  σε μια λιστα με ονομα tokens

3) ``stop_words = set(stopwords.words("english"))`` και  ``tokens = [word for word in tokens if word not in stop_words]``
Stopwords ειναι κοινες λεξεις της αλφαβητου που δεν εχουν καποια ιδιαιτερη σημασια( ειδικα για εφαρμογες οπως web scraping ) και καλο ειναι να αφαιρουνται
Η πρωτη εντολη `stopwords.words` ειναι μια μεθοδος που επιστρεφει μια λιστα απο stopwords( αναλογως της επιλογης της παραμετρου )... σε αυτη την περιπτωση περνουμε την λιστα και την κανουμε ενα set list( δηλαδη λιστα που δεν επιτρεπει την επαναληψη 2 ιδιων τιμων ) και την αποθηκευουμε στην μεταβλητη stop_words

Επειτα τρεχουμε το ηδη υπαρχων περιεχομενο της λιστας **tokens** που ειχαμε φτιαξει προηγουμενως χρησιμοποιωντας μια for loop για καθε ενα στοιχειο της λιστας. Δηλαδη η μεταβλητη **word** θα λαμβανει καθε μια τιμη της λιστας **tokens** και εφοσον το στοιχειο της **word** εκεινη την ωρα δεν εχει καποια ομοιοτητα με την **set** λιστα με ονομα **stop_words** ( δηλαδη να μην ειναι stop_word) τοτε θα αποθηκευεται στην λιστα tokens.

Σημαντικο οτι εδω η λιστα tokens γινεται overwrite

4) `lemmatizer = WordNetLemmatizer()` και `tokens = [lemmatizer.lemmatize(word) for word in tokens]`
Το lemmatization είναι η διαδικασία μετατροπής μιας λέξης στη βασική της μορφή (lemma), λαμβάνοντας υπόψη τη γραμματική της σημασία.
Δηλαδη την λεξη **τρεξιμο** θα την μετατρεψει ως **τρεχω**

Ο lemmatizer χρησιμοποιεί το WordNet ( το οποιο κατεβασαμε λιγο πιο πανω με την εντολη `nltk.download('wordnet')` ). Προκεται για ένα λεξικό με πληροφορίες για τις λέξεις ( μέρος του λόγου, λεξικό σημασίας )

Αρχικα δημιουργουμε ενα lemmatizer

Εφαρμοζουμε την μεθοδο **lemmatize** για καθε λεξη μεσα στην λιστα tokens 

In [ ]:
def preprocess_text(text):
    # Αφαίρεση ειδικών χαρακτήρων και αριθμών
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Tokenization
    tokens = word_tokenize(text)

    # Μετατροπή σε πεζά
    tokens = [word.lower() for word in tokens]

    # Αφαίρεση stop-words
    stop_words = set(stopwords.words("english"))  
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

# **Preprocessing**

1) `cleaned_text = []`
Δημιουργείται μια κενή λίστα με όνομα cleaned_text

2) `for item in column_data:`
Το column_data περιέχει όλα τα δεδομένα της στήλης Text του DataFrame data, σε μορφή λίστας Python.

Θυμιζω οτι η στηλη Text του DataFrame ειχε αποθηκευμενες ολες τις παραγραφους και η column data παιρνει αυτες τις παραγραφους για καθε γραμμη και τις ξανακανει σε λιστα python( απο csv που ηταν )

Δηλαδη κάθε στοιχείο της λίστας αντιστοιχεί σε καθε στοιχειο της στήλης Text στο DataFrame.


Το item παιρνει επαναληπτικα τις τιμες του column_data

3) `cleaned_text.append(preprocess_text(item))`
Για καθε τιμη( text ) της λιστας column_date( το item παρινει μια μια τις τιμες του column_date ), καλουμε την συναρτηση preprocess_text η οποια θα χειριστει την προεπεξεργασια(για τα επομενα βηματα)...δηλαδη εδω θα κανουμε tokenization

In [ ]:
cleaned_text = []
for item in column_data:
    cleaned_text.append(preprocess_text(item))

In [ ]:
from collections import defaultdict

# ***Δηλωση της συναρτησης create_inverted_index και λειτουργεια της***

**εχω εξηγησει εκτενως(στην αναφορα) τι ειναι ενα inverted index σε σχεση με μια απλη δομη δεδομενων**

Η συναρτηση καλειται απο την εκφραση `inverted_index = create_inverted_index(cleaned_text)` που ειναι μετα την συναρτηση(στην αρχη γινεται η δηλωση της, και μετα εκτελειται απο αυτο).

Οταν καλειται δεχεται ως παραμετρο το **documents** το οποιο ειναι στην ουσια η λιστα **cleaned_text** που δημιουργηθηκε μετα το tokenization που καναμε πιο πανω...δηλαδη το ***documents => cleaned_text => λιστα tokens( της συναρτησης preprocess_text )*** που εχει γινει ολη η διαδικασια του tokenization

1) `inverted_index = defaultdict(list)`

Για να δημιουργήσουμε ένα inverted index στην πραγματικότητα θα πρέπει να φτιάξουμε μία δομή δεδομένων που λειτουργεί ως λεξικό. 

**Δεν υπάρχει ήδη ετοιμη κλαση που να λειτουργεί ως inverted index**

Η defaultdict  κατάλληλα για αυτή την περίπτωση  διότι μας επιτρέπει  να φτιάξουμε μία δομή δεδομένων  που δέχεται λέξεις(  δηλαδή ένα λεξικό ) και για κάθε λέξη,  να αποθηκεύουμε πληροφορίες για αυτήν.

1. list in ο τύπος δεδομένων που θα αποθηκεύει η python( χρησιμοποίησει  η python  για κάθε λέξη )
2. defaultdict  είναι  μία ειδική κλάση της python που αντιπροσωπεύει λεξικό(  δηλαδή μία δομή δεδομένων που σε αυτή μπορούμε να αποθηκεύσουμε λέξεις )
Επομένως με αυτά τα δύο έχουμε φτιάξει μία δομή δεδομένων  που δέχεται λέξεις,  και για κάθε λέξη μπορούμε να έχουμε μία λίστα  που την αντιπροσωπεύει

Η defaultdict  έχει  την ικανότητα να φτιάχνει αυτή  κενές λίστες  για κάθε λέξη(  στοιχείο )  που θα εισάγουμε σε αυτήν και αυτό θα μας βοηθήσει ιδιαίτερα,  διότι σε ένα πρώτο στάδιο φτιάχνει αυτόματα  τον inverted index  από μόνη της. 

2) `for doc_id, tokens in enumerate(documents):`
Το **enumerate(documents)** μαζι με την for διατρέχει όλα τα έγγραφα στη λίστα documents και διαβαζει/επιστρεφει σε ποιο indexing βρισκομαστε( της λιστας documents ) και το περιεχομενο του εγγραφου.

Η λίστα documents θυμίζω ότι έχει σε κάθε θεση της,  έχει τα καθαρισμένα tokens

Επιστρέφεται ένα tuple τιμων:
1. Η τιμη του στοιχειου(indexing) της λιστας που επεξεργάζεται εκείνη την ώρα...αποθηκευεται στο doc_id
2. Tο περιεχόμενο της λιστας(  που είναι στην ουσία τα καθαρισμένα tokens )....**αποθηκευεται στην τοπικη λιστα tokens**

2) `for word in set(tokens):`
Φτιάχνουμε μία set λίστα όπου θα έχει περιεχόμενο όλα τα tokens(  που σημαίνει ότι δεν μπορούν να υπάρχουν και διπλότυπα )  και περιηγούμαστε σε αυτή  χρησιμοποιώντας μία for loop. 
   
3) `inverted_index[word].append(doc_id)`
Εφόσον η for loop  περγείται κάθε  token της λιστας set( που είναι στην ουσία η λίστα documents),  μπορούμε να  εκμεταλλευτούμε την ιδιότητα της defaultdict (  που είναι να προσθετει τα στοιχεία που ψάχνουμε όταν δεν υπάρχουν ηδη μέσα στη λίστα )  και παράλληλα να κάνουμε append σε καθε ενα απο αυτά  την  θέση του εγγράφου που βρίσκονται(doc_id)


Ετσι φτιάχνουμε αυτόματα (  με αυτόν τον έξυπνο τρόπο  που λειτουργεί η defaultdict)  το inverted index  και παράλληλα  προσθέτουμε στα  keywords  τι θέση που βρίσκονται στην λίστα documents

In [ ]:
def create_inverted_index(documents):
    inverted_index = defaultdict(list)  # Κάθε όρος αντιστοιχεί σε λίστα εγγράφων
    for doc_id, tokens in enumerate(documents):
        
        # Εισαγωγή λέξεων στη δομή
        for word in set(tokens):  # Χρησιμοποιούμε set για να αποφύγουμε επαναλήψεις
            inverted_index[word].append(doc_id)
    return inverted_index

In [ ]:
inverted_index = create_inverted_index(cleaned_text)

1) `inverted_index.items()`
Επιστρέφει όλα τα ζευγάρια κλειδιών και τιμών (key-value pairs) από το λεξικό inverted_index.

2) `list(inverted_index.items())[:5]`
Μετατρέπει τα αντικείμενα που επιστρέφει το inverted_index.items() σε λίστα και με το [:5], παίρνει τα πρώτα 5 στοιχεία από αυτή τη λίστα.
Αν το inverted_index παραπάνω απο 5 λέξεις τοτε η εντολή [:5] θα περιορίσει την επανάληψη στις πρώτες 5 λέξεις.

3) `print(f"{word}: {doc_ids}")`
Χρησιμοποιεί τη μορφοποίηση f-string της Python για να εμφανίσει τη λέξη και τα IDs των εγγράφων όπου εμφανίζεται.

In [ ]:
for word, doc_ids in list(inverted_index.items())[:5]:
    print(f"{word}: {doc_ids}")

# ***Importing βιβλιοθηκης json***

Αρχικα Κανουμε import την βιβλιοθηκη json ( επεξηγηση για το τι ειναι, υπαρχει στην αναφορα )

1) `open("inverted_index.json", "w", encoding="utf-8")`
Ανοίγει (ή δημιουργεί αν δεν υπαρχει ηδη) ένα αρχείο με το όνομα inverted_index.json στη λειτουργία write ( δηλαδη μπορουμε να το επεξεργαστουμε )

2) `json.dump()`
Αποθηκεύει δεδομένα σε μορφή JSON σε ένα αρχείο.

3) `ensure_ascii=False`
Διασφαλίζει ότι τα μη-ASCII δεδομένα θα αποθηκευτούν σωστά, αντί να κωδικοποιηθούν σε χαρακτήρες Unicode

4) `indent=4`
Κανουμε πιο ευαναγνστα τα δεδομενα, βαζοντας κενα πριν απο καθε δεδομενα( συγκεκριμενα εδω βαζουμε 4 κενα )


To αρχειο ανοιγει με την ονομασια f

In [ ]:
import json
# Save the inverted index to a JSON file
with open("inverted_index.json", "w", encoding="utf-8") as f:
    json.dump(inverted_index, f, ensure_ascii=False, indent=4)

# ***Χρηση Inverted Index*** 

Αρχικα ανοιγουμε το αρχειο σε λειτουργεια read με `encoding="utf-8` ετσι ωστε να μπορουν να διαβαζονται σωστα οι ascii χαρακτηρες
Το αρχειο ανοιγει με την ονομασια f

`inverted_index = json.load(f)`
Φορτώνει τα δεδομένα JSON από το αρχείο f και μετατρεπει το αρχειο json σε μια δομή δεδομένων της Python
1. Αν το JSON είναι ένα λεξικό =>  γίνεται dict
2. Αν το JSON είναι μια λίστα => γίνεται list

In [ ]:
with open("inverted_index.json", "r", encoding="utf-8") as f:
    inverted_index = json.load(f)

In [ ]:
csv_file_path = "wikipedia_data.csv"

# Load the CSV file into a DataFrame
data = pd.read_csv(csv_file_path, header=0)    
#it has the urls of the docs
urls = data['URL'].tolist() 
documents = data['Text'].tolist()

# ***Δηλωση της συναρτησης preprocess_query και λειτουργεια της***
**πολλα βηματα εδω εχουν εξηγηθει με παρομοιο τροπο προηγουμενως**

Αυτη η συναρτηση επεξεργαζεται ενα query

1) Αρχικα δημιουργει ενα **set αντικειμενο** με τα stopwords της αγγλικης γλωσσας( οπως ειχα κανει/εξηγησει και παραπανω )
2) Επειτα στην λιστα των stop words αφαιρουμε τις λεξεις:
   1. **and**
   2. **or**
   3. **not**

3) Κανουμε απλο tokenize το query
4) Μετατρεπουμε καθε λεξη σε πεζα γραμματα ( με `token.lower()`) και αφαιρουμε οτι δεν ειναι γραμμα ( λογω του regex του )
5) Τελος στο tokenized query, το περναμε απο μια for Loop και αφαιρουμε τις λεξεις που **ΠΛΕΟΝ θεωρουνται stop_words**

In [ ]:
def preprocess_query(query):
    stop_words = set(stopwords.words("english")) 
    #it should not remove AND OR NOT
    stop_words = [word for word in stop_words if word not in ['and', 'or', 'not']]
    tokens = word_tokenize(query)
    tokens = [re.sub(r"[^a-zA-Z]", "", token.lower()) for token in tokens]
    tokens = [word for word in tokens if word and word not in stop_words]
    return tokens

# ***Δηλωση των συναρτησεων and,or,not***

Πως λειτουργει η boolean retrival

Το posting 1 και 2 ειναι λιστες αριθμων. Πιο συγκεκριμενα
1. posting 1: Eίναι η λίστα η περιέχει το συνολικό αριθμό των εγγράφων( δηλαδή μία λίστα ακεραιων οπου κάθε θέση της ειναι ενα εγγραφο )
2. posting 2: Είναι η λίστα που περιεχει τα εγγραφα( σαν αριθμους ) που εμφανιστηκε το token που θελουμε να επεξεργαστουμε

Αρα ειναι 2 λιστες ακεραιων και η boolean Λογικη δουλευει ως εξης: 

### AND

Στην λογική πράξη AND,  το αποτέλεσμα  είναι μία τελική λίστα  όπου  θα έχει ως περιεχόμενο τους αριθμούς  που είναι κοινοι και στις 2 λιστες.


### OR 

Συνδυάζονται τα στοιχεία από δύο σύνολα, χωρίς διπλότυπα.
Διοτι η OR σε μια τετοια περιπτωση θα δεχεται τα παντα


### NOT 

Αφαιρούνται από το πρώτο σύνολο τα στοιχεία του δεύτερου.

In [ ]:
def boolean_and(postings1, postings2):
    return list(set(postings1) & set(postings2))

def boolean_or(postings1, postings2):
    return list(set(postings1) | set(postings2))

def boolean_not(postings, total_docs):
    return list(set(total_docs) - set(postings))

# ***Δηλωση της search και η λειτουργεια της***
 
### query
Προκειται για ένα query το οποίο έχει δοθεί  από εμένα ως `query = "crawler AND bot"` ΑΡΓΟΤΕΡΑ ΣΤΟΝ ΚΩΔΙΚΑ
ΔΙΟΤΙ ΠΡΩΤΑ ΔΗΛΩΝΩ ΟΛΕΣ ΤΙΣ ΣΥΝΑΡΤΗΣΕΙΣ
θα λειτουργησουν συνεργατικα

### inverted_index
Η δομη inverted index που έχει δημιουργηθεί προηγουμένως από τον κώδικα

### total_docs
Παιρνει την τιμη της με βαση αυτη την εντολη  `total_docs = data.shape[0]`, η οποια επιστρεφει το πληθος των γραμμων στο dataframe που ειχαμε φτιαξει.

Ο λόγος που συμβαίνει αυτό είναι γιατί  κάνουμε χρήση της βιβλιοθήκης pandas  και πιο συγκεκριμένα της μεθόδου shape  η οποια επιστρέφει ένα tuple τιμων.

1. shape[0] Ο αριθμός των γραμμών μιας λιστας 
2. shape[1] Ο αριθμός των στηλών μιας λιστας

Επομένως εφοσον κάνουμε χρήση αυτής της μεθόδου  στην λίστα data και επιλέγουμε να πάρουμε το πρώτο στοιχείο του tuple( shape[0] ),  τότε θα μας επιστραφεί  ο συνολικός αριθμός των γραμμών του dataframe

## `query_tokens = preprocess_query(query)`

Παίρνουμε το προηγούμενο query και το περνάμε από διαδικασια tokenization, αποθηκεύοντας το αποτέλεσμα( λιστα )  στη μεταβλητή query_tokens


## `results = set(range(total_docs))`

Η συνάρτηση range δημιουργεί μία ακολουθία αριθμών  και δέχεται ως όρισμα έναν ακέραιο...γενικα  σαν συνάρτηση δέχεται μόνο ακεραιους). Κανουμε χρηση μονο της λειτουργιας της πρωτης παραμετρου της( start ) διοτι αυτη η συναρτηση εχει 3 παραμετρους.

Ο ακέραιος που δέχεται ως ορισμα  είναι ο αριθμός των γραμμών του data frame που είχαμε δημιουργήσει( ακριβως απο πανω εξηγω γιατι )

Τέλος φτιάχνουμε μία λίστα set(  έχω εξηγήσει πολλές φορές τι ειναι αυτο μεχρι τωρα )  έτσι ώστε να έχουμε μία λίστα με τον συνολικό αριθμό των εγγράφων χωρις διπλοτυπα.

## `operation = "AND"`

Επιλεγω αρχικα την boolean τιμή "and" ( για να δηλωσω και την μεταβλητη operation ως boolean...αλλα αναλογως το query μας, αυτο μπορει να μην εχει ιδιατερη σημασια )

## Επεξηγηση Loop

Aφού έχουμε δημιουργήσει την λίστα query_tokens(  που έχει προέλθει από το αρχικό μας query με tokenization ),  κάνουμε προσπέλαση σε κάθε στοιχείο αυτής της λίστας χρησιμοποιώντας μία for loop.

Mε τη χρήση της μεθόδου `upper` μετατρέπουμε με το token  της λιστας που επεξεργαζόμαστε εκείνη την στιγμή σε κεφαλαια,  και ελέγχουμε  αν είναι  κάποια από τις τιμές "AND" "OR" "NOT" 


**Αν ειναι τοτε,  το αποθηκεύουμε στην μεταβλητη operation  έτσι ώστε να το χρησιμοποιήσουμε αργότερα.** 

**Αν δεν είναι αναζητάμε στο inverted index αυτό το token  και την λίστα του.**


Τελος αναλόγως της τιμης του operation,  θα καλέσουμε και την αντίστοιχη συνάρτηση για boolean retrieval.

**Η λογικη της boolean retrival, την εξηγω απο πανω στο δικο της μερος**



In [ ]:
def search(query, inverted_index, total_docs):
    # Χωρισμός σε λέξεις και προεπεξεργασία
    query_tokens = preprocess_query(query)
    
    # Ανάλυση ερωτήματος για Boolean λειτουργίες
    results = set(range(total_docs))  # Όλα τα έγγραφα αρχικά
    operation = "AND"  # Default λειτουργία
    
    for token in query_tokens:
        if token.upper() in ["AND", "OR", "NOT"]:
            operation = token.upper()
        else:
            postings = inverted_index.get(token, [])
            
            if operation == "AND":
                results = boolean_and(results, postings)
            elif operation == "OR":
                results = boolean_or(results, postings)
            elif operation == "NOT":
                results = boolean_not(postings, range(total_docs))
    
    return sorted(results)

# ***Δηλωση της compute_tf και η λειτουργεια της***

Η συνάρτηση compute_tf υπολογίζει τη συχνότητα όρου (Term Frequency - TF) μιας συγκεκριμένης λέξης μέσα σε ένα έγγραφο 
Εδω θα υπολογισουμε ποσες φορες εμφανιζεται μια λεξη στις παγραφους ενος html αρχειου( αυτο θα γινει για καθε html αρχειο ) και θα το συγκρινουμε με το συνολικο πληθος των λεξεων του html 

Ετσι θα βγαλουμε την μαθηματικη σχεση της συχνοτητας

## Παραμετροι 

`doc`: Ειναι το documents[doc_id] που καλειται αργοτερα

Θυμιζω οτι η λιστα documents εχει δημιουργηθει απο το dataframe των παραγραφων του καθε html αρχειου οπου καθε θεση της λιστας περιεχει ολες τις παραγραφους ενος html αρχειου.

`word`: Η λέξη για την οποία θέλουμε να υπολογίσουμε τη συχνότητα εμφάνισης στο κείμενο του εγγράφου 

1) `doc.split()`: Διαχωρίζει τις παραγραφους( που εχει το dataframe για ενα συγκεκριμενο html) σε μια λίστα από λέξεις 
2) `words.count(word)`: Μετράει πόσες φορές εμφανίζεται η λέξη word στη λίστα words( η οποια ειναι το αποτελεσμα της doc.split απο πανω )
3) `len(words)`: Υπολογίζει τον συνολικό αριθμό των λέξεων στη λίστα words.
4) `word_count / total_words`: Μαθηματικη εκφραση που υπλογιζει την συχνοτητα εμφανισης της λεξης

In [ ]:
def compute_tf(doc, word):
    words = doc.split()
    word_count = words.count(word)
    total_words = len(words)
    tf = word_count / total_words
    return tf

In [ ]:
import math

# ***Δηλωση της compute_idf και η λειτουργεια της***

Η συνάρτηση compute_idf υπολογίζει την αντίστροφη συχνότητα εγγράφου ( Inverse Document Frequency ) μιας λέξης 

Δηλαδη μετράει πόσο "σπάνια" είναι μια λέξη σε ολόκληρη τη συλλογή εγγράφων

1) `inverted_index.get(word, [])`: Βρίσκει τη λίστα των εγγράφων που περιέχουν τη λέξη word.
Επειτα ελέγχουμε αν η λεξη δεν εμφανιζεται σε κανεναν εγγραφο, και αν ισχυει επιστρεφουμε την τιμη 0...διαφορετικα υπολογιζουμε την τιμη 

In [ ]:
def compute_idf(inverted_index, word, total_docs):
    doc_freq = len(inverted_index.get(word, []))
    if doc_freq == 0:
        return 0
    idf = math.log(total_docs / doc_freq)
    return idf

# ***Δηλωση της compute_tfidf και η λειτουργεια της***

Η συνάρτηση compute_tfidf υπολογίζει τη συχνότητα όρου-αντίστροφη συχνότητα εγγράφου (TF-IDF) μιας λέξης (word) σε ένα συγκεκριμένο έγγραφο (doc_id)
Για τον υπολογισμο του, συνδυαζονται οι τιμες του tf και idf 

In [ ]:
def compute_tfidf(doc_id, word, documents, inverted_index):
    total_docs = len(documents)
    
    # Υπολογισμός TF για το έγγραφο
    tf = compute_tf(documents[doc_id], word)
    
    # Υπολογισμός IDF για τη λέξη
    idf = compute_idf(inverted_index, word, total_docs)
    
    # Υπολογισμός TF-IDF
    tfidf = tf * idf
    return tfidf

In [ ]:
import numpy as np

# ***Δηλωση της search_tf_idf και η λειτουργεια της***

Η συνάρτηση search_tf_idf εκτελεί αναζήτηση TF-IDF σε μια συλλογή εγγράφων για ένα συγκεκριμένο ερώτημα
Χρησιμοποιεί τη συνάρτηση search για να επιστρέψει τα σχετικά έγγραφα (τα έγγραφα που περιέχουν τις λέξεις του ερωτήματος)

Xρησιμοποιεί τη συνάρτηση preprocess_query για να επεξεργαστεί το ερώτημα και να το μετατρέψει σε tokens, αφαιρωντας stop_words εκτος των boolean λεξεων που βγαλαμε απο το συνολο των stopwords

Επειτα για καθε λεξη αν δεν ειναι boolean, τοτε Υπολογίζει το TF-IDF score της λέξης για κάθε έγγραφο στα result_docs και αποθηκεύει τα TF-IDF scores της λέξης στη λίστα scores

Μετατρέπει το q_scores σε NumPy array για εύκολους υπολογισμούς
Υπολογίζει τον μέσο όρο (mean) των TF-IDF scores κάθε λέξης για κάθε έγγραφο

In [ ]:
def search_tf_idf(query, inverted_index, documents):
    total_docs = len(documents)
    result_docs = search(query, inverted_index, total_docs)

    query_tokens = preprocess_query(query)
    q_scores = []
    for q in query_tokens:
        if q not in ['and', 'or', 'not']:
            scores = []
            for i in result_docs:
                scores.append(compute_tfidf(i,q,documents,inverted_index))
            q_scores.append(scores)
    
    q_scores = np.array(q_scores)        
    final_scores = np.mean(q_scores,axis=0)
    
    
    sorted_terms, sorted_scores = zip(*sorted(zip(result_docs, final_scores), key=lambda x: x[1], reverse=True))
    
    # Convert back to lists 
    sorted_terms = list(sorted_terms)
    sorted_scores = list(sorted_scores)
    return(sorted_terms, sorted_scores)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ***Δηλωση της vector_space_retrieve και η λειτουργεια της***

**στην αναφορά εξηγώ εκτενως τι είναι το VSR μοντελο και πως λειτουργει**

## Παραμετροι 

`query`: Η ερωτηση...δηλαδη αυτο που θελω να ψαξω στα εγγραφα
`inverted_index`: Το ανάστροφο ευρετήριο απο πριν
`documents`: Καθε θεση αυτης της λιστας εχει ολες τις παραγραφους του καθε html αρχειου αντιστοιχα

## Εντολες 

1. `total_docs = len(documents)`: Επιστρεφει το πληθος των στοιχειων της λιστας documents 
2. `result_docs = search(query, inverted_index, total_docs)`: Εκμεταλευομαστε την συναρτηση search διοτι θα μας επιστρεψει ποια εγγραφα περιεχουν τις λεξεις του query

3. `vectorizer = TfidfVectorizer()`: μετατρέπει κείμενο (έγγραφα) σε μαθηματικά διανύσματα TF-IDF χρησιμοποιωντας την βιβλιοθήκη scikit-learn
4. `query_vector = vectorizer.transform([query])` : μετατρεπει το query( ερωτηση ) σε διανυσμα χρησιμοποιωντας την βιβλιοθήκη scikit-learn
5. `query_tfidf = query_vector.toarray()`: μετατρεπει το διανυσμα του query σε πινακα array ( γιατι απο μονο του θα ηταν μια τιμη tf idf )
6. `document_vectors = tfidf_matrix[result_docs]` : επιλέγει τα TF-IDF διανύσματα μόνο για τα έγγραφα που είναι υποψήφια για σύγκριση, δηλαδή αυτά που επέστρεψε η search ως σχετικά με το query.
7. `similarities = cosine_similarity(query_tfidf, document_vectors)`: υπολογίζει την cosine similarity του διανύσματος της ερώτησης και των διανυσμάτων των εγγράφων.

Στο τελος ταξινομουμε τα αποτελέσματα της cosine similarity με φθίνουσα σειρά, συνδυάζοντας τα έγγραφα με τις βαθμολογίες συσχέτισης (similarities[0]). Τελικά, επιστρέφει δύο ξεχωριστές λίστες:

1. Τα IDs των εγγράφων (sorted_terms).
2. Τις αντίστοιχες βαθμολογίες cosine similarity (sorted_scores).

Στο τελος μετατρεπουμε το tuple τιμων που επιστεψε η zip, πισω σε μορφη λιστας

In [ ]:
def vector_space_retrieve(query, inverted_index, documents):
    total_docs = len(documents)
    result_docs = search(query, inverted_index, total_docs)
    
    
    # Calculate the cosine similarity between the query and the documents
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    query_vector = vectorizer.transform([query])
    
    
    # Get the TF-IDF vector of the query (it will be a sparse matrix)
    query_tfidf = query_vector.toarray()

    # Compute cosine similarity between the query and all documents in the result set
    document_vectors = tfidf_matrix[result_docs]
    
    # Compute cosine similarity of query with each document in the result set
    similarities = cosine_similarity(query_tfidf, document_vectors)
    
    
    sorted_terms, sorted_scores = zip(*sorted(zip(result_docs, list(similarities[0])), key=lambda x: x[1], reverse=True))
    
    # Convert back to lists 
    sorted_terms = list(sorted_terms)
    sorted_scores = list(sorted_scores)
    
    return(sorted_terms, sorted_scores)

In [ ]:
from collections import Counter

### ***ΕΔΩ ΜΕ ΕΠΙΑΣΕ ΤΕΝΟΝΤΙΤΙΔΑ...ΟΠΩΣ ΓΡΑΦΩ ΣΤΟ WORD ΥΠΟΦΕΡΩ ΑΠΟ ΜΕΓΑΛΟ ΘΕΜΑ ΜΕ ΚΗΛΗ ΣΤΟ ΑΥΧΕΝΑ + ΚΑΡΠΙΑΙΟ***
ΔΕΝ ΜΠΟΡΩ ΝΑ ΓΡΑΨΩ ΑΛΛΟ ΤΗΝ ΕΠΕΞΗΓΗΣΗ ΑΠΟ ΕΔΩ ΚΑΙ ΠΕΡΑ

In [ ]:
def build_inverted_index_freq(documents):
        inverted_index = {}
        for doc_id, doc in enumerate(documents):
            terms = doc.split()
            term_freq = Counter(terms)
            for term, freq in term_freq.items():
                if term not in inverted_index:
                    inverted_index[term] = []
                inverted_index[term].append((doc_id, freq))
        return inverted_index

In [ ]:
def calculate_bm25(query, inverted_index, documents):
    #params
    k1=1.5
    b=0.75
    
    doc_lengths = [len(doc.split()) for doc in documents]
    avg_doc_length = sum(doc_lengths) / len(documents)
    
    #calc IDF
    idf = {}
    total_docs = len(documents)
    for term in inverted_index:
        doc_freq = len(inverted_index[term])
        idf[term] = math.log((total_docs - doc_freq + 0.5) / (doc_freq + 0.5) + 1.0)
    
    total_docs = len(documents)
    result_docs = search(query, inverted_index, total_docs)
    
    scores = [0] * len(result_docs)  # Score for each document
    query_terms = preprocess_query(query)
    for term in query_terms:
        if term not in ['and', 'or', 'not']:
            if term in inverted_index:
                idf_t = idf.get(term, 0)
                for i, doc_id in enumerate(result_docs):
                    word_list = documents[doc_id].split()
                    term_freq = word_list.count(term)
                    doc_length = doc_lengths[doc_id]
                    tf = term_freq
                    score = idf_t * tf * (k1 + 1) / (tf + k1 * (1 - b + b * doc_length / avg_doc_length))
                    scores[i] += score
                    
    sorted_terms, sorted_scores = zip(*sorted(zip(result_docs, scores), key=lambda x: x[1], reverse=True))
    
    # Convert back to lists 
    sorted_terms = list(sorted_terms)
    sorted_scores = list(sorted_scores)
    
    return(sorted_terms, sorted_scores)                

In [ ]:
query = "crawler AND bot"
total_docs = data.shape[0]  # Συνολικός αριθμός εγγράφων

# Αναζήτηση Boolean Retrieval
result_docs = search(query, inverted_index, total_docs)
print("Αποτελέσματα Boolean Retrieval:")
for i in result_docs:
    print(urls[i])

# Αναζήτηση TF-IDF
sorted_terms, sorted_scores = search_tf_idf(query, inverted_index, documents)

print("Αποτελέσματα TF-IDF:")
for i in range(len(sorted_terms)):
    print("%s \t %.6f" %( urls[sorted_terms[i]], sorted_scores[i]))

# Αναζήτηση vector space retrieve
sorted_terms, sorted_scores = vector_space_retrieve(query, inverted_index, documents)

print("Αποτελέσματα vector space retrieve:")
for i in range(len(sorted_terms)):
    print("%s  \t %.6f" %( urls[sorted_terms[i]], sorted_scores[i]))


# Αναζήτηση bm25
sorted_terms, sorted_scores = calculate_bm25(query, inverted_index, documents)

print("Αποτελέσματα BM25:")
for i in range(len(sorted_terms)):
    print("%s  \t %.6f" %( urls[sorted_terms[i]], sorted_scores[i]))
    

# EVALUATION - USE NEWSGROUP DATA

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

def scores(retrieved_results, relevant_results, M):
    # Δημιουργία των δυαδικών λιστών (true/false) για τα αποτελέσματα
    # Αν το έγγραφο είναι σχετικό, το σημειώνουμε ως 1, αλλιώς ως 0
    y_true = [1 if doc in relevant_results else 0 for doc in range(M)]  # Σχετικά έγγραφα
    y_pred = [1 if doc in retrieved_results else 0 for doc in range(M)]  # Επιστρεφόμενα έγγραφα

    # Υπολογισμός των μετρικών
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    map_score = average_precision_score(y_true, y_pred)

    # Εκτύπωση αποτελεσμάτων
    print(f"Ακρίβεια (Precision): {precision:.4f}")
    print(f"Ανάκληση (Recall): {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"Μέση Ακρίβεια (MAP): {map_score:.4f}")
    
    
# Load the dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data
labels = newsgroups.target
categories = newsgroups.target_names



import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

query = "space shuttle launch"
total_docs = len(documents)  # Συνολικός αριθμός εγγράφων
relevant_results = np.where(labels == categories.index('sci.space'))
relevant_results = list(relevant_results[0])    


top_k = 10
# Αναζήτηση Boolean Retrieval
sorted_terms = search(query, inverted_index, total_docs)
print("Αποτελέσματα Boolean Retrieval:")
for i in range(top_k):
    print(sorted_terms[i])

retrieved_results = list(sorted_terms[:top_k])
scores(retrieved_results, relevant_results, total_docs)

# Αναζήτηση TF-IDF
sorted_terms, sorted_scores = search_tf_idf(query, inverted_index, documents)
top_k = 10
print("Αποτελέσματα TF-IDF:")
for i in range(top_k):
    print("%s %.6f" %( sorted_terms[i], sorted_scores[i]))

retrieved_results = sorted_terms[:top_k]
scores(retrieved_results, relevant_results, total_docs)

# Αναζήτηση vector space retrieve
sorted_terms, sorted_scores = vector_space_retrieve(query, inverted_index, documents)

print("Αποτελέσματα vector space retrieve:")
for i in range(top_k):
    print("%s %.6f" %( sorted_terms[i], sorted_scores[i]))

retrieved_results = sorted_terms[:top_k]
scores(retrieved_results, relevant_results, total_docs)

# Αναζήτηση bm25
sorted_terms, sorted_scores = calculate_bm25(query, inverted_index, documents)

print("Αποτελέσματα BM25:")
for i in range(top_k):
    print("%s %.6f" %( sorted_terms[i], sorted_scores[i]))

retrieved_results = sorted_terms[:top_k]
scores(retrieved_results, relevant_results, total_docs)